## Module 1 Homework (2025 cohort)

In this homework, we're going to download finance data from various sources and make simple calculations or analysis.


### Question 1: [Index] S&P 500 Stocks Added to the Index

**Which year had the highest number of additions?**

Using the list of S&P 500 companies from Wikipedia's [S&P 500 companies page](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies), download the data including the year each company was added to the index.
* Create a DataFrame with company tickers, names, and the year they were added.
* Extract the year from the addition date and calculate the number of stocks added each year.
* Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)? Write down this year as your answer (the most recent one, if you have several records).

*Context*: 
> "Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 ([Nasdaq article](https://www.nasdaq.com/articles/sp-500-reshuffle-dash-tko-expe-wsm-join-worth-buying)).

*Additional*: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.

In [11]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url, header=[0,1])
changes_df = tables[1]

# Flatten columnas MultiIndex para facilitar el manejo
changes_df.columns = ['_'.join(col).strip() for col in changes_df.columns.values]

# Filtrar solo filas con adiciones (donde Added_Ticker no sea nulo)
additions = changes_df[changes_df['Added_Ticker'].notna()]

# Convertir la fecha a datetime
additions['Date_Date'] = pd.to_datetime(additions['Date_Date'], errors='coerce')

# Extraer año
additions['Year'] = additions['Date_Date'].dt.year

# Excluir 1957
additions = additions[additions['Year'] != 1957]

# Contar adiciones por año
additions_per_year = additions['Year'].value_counts().sort_index()

# Año con más adiciones (último si hay empate)
most_additions_year = additions_per_year[additions_per_year == additions_per_year.max()].index[-1]

print("Year with the most additions:", most_additions_year)
print("Additions per year:")
print(additions_per_year)



Year with the most additions: 2016
Additions per year:
Year
1976     2
1994     1
1997     1
1998     3
1999     3
2000     7
2003     1
2005     2
2006     1
2007    11
2008     8
2009    12
2010    11
2011    19
2012    18
2013    19
2014    16
2015    28
2016    30
2017    28
2018    23
2019    24
2020    17
2021    19
2022    17
2023    15
2024    16
2025     5
Name: count, dtype: int64


C:\Users\Sandra\AppData\Local\Temp\ipykernel_5280\3271340431.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  additions['Date_Date'] = pd.to_datetime(additions['Date_Date'], errors='coerce')
C:\Users\Sandra\AppData\Local\Temp\ipykernel_5280\3271340431.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  additions['Year'] = additions['Date_Date'].dt.year


Which year had the highest number of additions to the S&P 500 index? (2 points)
* **2017 (28)**
* 2019 (24)
* 2008 (8)
* 2023 (15)

### Question 2. [Macro] Indexes YTD (as of 1 May 2025)

**How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?**

Using Yahoo Finance World Indices data, compare the year-to-date (YTD) performance (1 January-1 May 2025) of major stock market indexes for the following countries:
* United States - S&P 500 (^GSPC)
* China - Shanghai Composite (000001.SS)
* Hong Kong - HANG SENG INDEX (^HSI)	
* Australia - S&P/ASX 200 (^AXJO)
* India - Nifty 50 (^NSEI)
* Canada - S&P/TSX Composite (^GSPTSE)
* Germany - DAX (^GDAXI)
* United Kingdom - FTSE 100 (^FTSE)
* Japan - Nikkei 225 (^N225)
* Mexico - IPC Mexico (^MXX)
* Brazil - Ibovespa (^BVSP)

*Hint*: use start_date='2025-01-01' and end_date='2025-05-01' when downloading daily data in yfinance

Context: 
> [Global Valuations: Who's Cheap, Who's Not?](https://simplywall.st/article/beyond-the-us-global-markets-after-yet-another-tariff-update) article suggests "Other regions may be growing faster than the US and you need to diversify."

Reference: Yahoo Finance World Indices - https://finance.yahoo.com/world-indices/

*Additional*: How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods? Do you see the same trend?
Note: For simplicity, ignore currency conversion effects.)


In [15]:
import yfinance as yf
import pandas as pd

indices = {
    "S&P 500 (US)": "^GSPC",
    "Shanghai Composite (China)": "000001.SS",
    "HANG SENG INDEX (HK)": "^HSI",
    "S&P/ASX 200 (Australia)": "^AXJO",
    "Nifty 50 (India)": "^NSEI",
    "S&P/TSX Composite (Canada)": "^GSPTSE",
    "DAX (Germany)": "^GDAXI",
    "FTSE 100 (UK)": "^FTSE",
    "Nikkei 225 (Japan)": "^N225",
    "IPC Mexico": "^MXX",
    "Ibovespa (Brazil)": "^BVSP"
}

start_date = "2025-01-01"
end_date = "2025-05-01"

# Download data (Close prices)
data = yf.download(list(indices.values()), start=start_date, end=end_date)['Close']

# Calculate YTD return for each index ensuring no NaNs are used
returns = {}
for ticker in data.columns:
    series = data[ticker].dropna()
    if len(series) > 1:
        ret = (series.iloc[-1] / series.iloc[0]) - 1
        returns[ticker] = ret
    else:
        returns[ticker] = float('nan')  # Not enough data

# Create DataFrame with results
returns_df = pd.DataFrame({
    'Index': [k for k in indices.keys()],
    'Ticker': [v for v in indices.values()],
    'YTD Return': [returns.get(ticker, float('nan')) for ticker in indices.values()]
})

returns_df = returns_df.sort_values(by='YTD Return', ascending=False).reset_index(drop=True)

# S&P 500 return
ret_snp = returns_df.loc[returns_df['Ticker'] == '^GSPC', 'YTD Return'].values[0]

# Indexes with better return than S&P 500
better_than_snp = returns_df[returns_df['YTD Return'] > ret_snp]

print("\nYTD Returns (Jan 1, 2025 - May 1, 2025):")
print(returns_df)

print(f"\nNumber of indexes with better return than the S&P 500: {len(better_than_snp)}")
print("\nIndexes that outperform the S&P 500:")
print(better_than_snp[['Index', 'YTD Return']])



[*********************100%***********************]  11 of 11 completed


YTD Returns (Jan 1, 2025 - May 1, 2025):
                         Index     Ticker  YTD Return
0                   IPC Mexico       ^MXX    0.130494
1         HANG SENG INDEX (HK)       ^HSI    0.127200
2            Ibovespa (Brazil)      ^BVSP    0.124387
3                DAX (Germany)     ^GDAXI    0.123464
4                FTSE 100 (UK)      ^FTSE    0.028426
5             Nifty 50 (India)      ^NSEI    0.024904
6   Shanghai Composite (China)  000001.SS    0.005048
7   S&P/TSX Composite (Canada)    ^GSPTSE   -0.002261
8      S&P/ASX 200 (Australia)      ^AXJO   -0.009145
9                 S&P 500 (US)      ^GSPC   -0.051033
10          Nikkei 225 (Japan)      ^N225   -0.082979

Number of indexes with better return than the S&P 500: 9

Indexes that outperform the S&P 500:
                        Index  YTD Return
0                  IPC Mexico    0.130494
1        HANG SENG INDEX (HK)    0.127200
2           Ibovespa (Brazil)    0.124387
3               DAX (Germany)    0.123464
4   

How many indexes have better year-to-date returns than the US S&P500 as of May 1, 2025? (3 points)
* 5
* 7
* **9**
* 8

### Question 3. [Index] S&P 500 Market Corrections Analysis


**Calculate the median duration (in days) of significant market corrections in the S&P 500 index.**

For this task, define a correction as an event when a stock index goes down by **more than 5%** from the closest all-time high maximum.

Steps:
1. Download S&P 500 historical data (1950-present) using yfinance
2. Identify all-time high points (where price exceeds all previous prices)
3. For each pair of consecutive all-time highs, find the minimum price in between
4. Calculate drawdown percentages: (high - low) / high × 100
5. Filter for corrections with at least 5% drawdown
6. Calculate the duration in days for each correction period
7. Determine the 25th, 50th (median), and 75th percentiles for correction durations

*Context:* 
> * Investors often wonder about the typical length of market corrections when deciding "when to buy the dip" ([Reddit discussion](https://www.reddit.com/r/investing/comments/1jrqnte/when_are_you_buying_the_dip/?rdt=64135)).

> * [A Wealth of Common Sense - How Often Should You Expect a Stock Market Correction?](https://awealthofcommonsense.com/2022/01/how-often-should-you-expect-a-stock-market-correction/)

*Hint (use this data to compare with your results)*: Here is the list of top 10 largest corrections by drawdown:
* 2007-10-09 to 2009-03-09: 56.8% drawdown over 517 days
* 2000-03-24 to 2002-10-09: 49.1% drawdown over 929 days
* 1973-01-11 to 1974-10-03: 48.2% drawdown over 630 days
* 1968-11-29 to 1970-05-26: 36.1% drawdown over 543 days
* 2020-02-19 to 2020-03-23: 33.9% drawdown over 33 days
* 1987-08-25 to 1987-12-04: 33.5% drawdown over 101 days
* 1961-12-12 to 1962-06-26: 28.0% drawdown over 196 days
* 1980-11-28 to 1982-08-12: 27.1% drawdown over 622 days
* 2022-01-03 to 2022-10-12: 25.4% drawdown over 282 days
* 1966-02-09 to 1966-10-07: 22.2% drawdown over 240 days

In [23]:
import yfinance as yf
import pandas as pd

ticker = "^GSPC"
start_date = "1950-01-01"
end_date = "2025-05-01"

data = yf.download(ticker, start=start_date, end=end_date)
prices = data['Close']

# Make sure prices is a Series (not a DataFrame)
if isinstance(prices, pd.DataFrame):
    prices = prices.iloc[:, 0]

all_time_highs = prices[prices == prices.cummax()].index
ath_list = list(all_time_highs)

corrections = []

for i in range(len(ath_list) - 1):
    start = ath_list[i]
    end = ath_list[i+1]

    high_price = prices.at[start]
    min_price = prices.loc[start:end].min()
    drawdown_pct = (high_price - min_price) / high_price * 100

    if drawdown_pct >= 5:
        duration_days = (end - start).days
        corrections.append({
            "start": start,
            "end": end,
            "drawdown_pct": drawdown_pct,
            "duration_days": duration_days
        })

corrections_df = pd.DataFrame(corrections)
percentiles = corrections_df['duration_days'].quantile([0.25, 0.5, 0.75])

print("Number of corrections (>=5% drawdown):", len(corrections_df))
print("\nCorrection durations percentiles (in days):")
print(percentiles)

print("\nTop 5 corrections sorted by drawdown:")
print(corrections_df.sort_values(by='drawdown_pct', ascending=False).head())



[*********************100%***********************]  1 of 1 completed

Number of corrections (>=5% drawdown): 71

Correction durations percentiles (in days):
0.25     56.0
0.50     94.0
0.75    213.5
Name: duration_days, dtype: float64

Top 5 corrections sorted by drawdown:
        start        end  drawdown_pct  duration_days
56 2007-10-09 2013-03-28     56.775388           1997
54 2000-03-24 2007-05-30     49.146948           2623
24 1973-01-11 1980-07-17     48.203593           2744
22 1968-11-29 1972-03-06     36.061641           1193
65 2020-02-19 2020-08-18     33.924960            181


Median duration (in days) of significant market corrections in the S&P 500 index (3 points)
* 39
* 113
* **95**
* 74

### Question 4.  [Stocks] Earnings Surprise Analysis for Amazon (AMZN)


**Calculate the median 2-day percentage change in stock prices following positive earnings surprises days.**

Steps:
1. Load earnings data from CSV ([ha1_Amazon.csv](ha1_Amazon.csv)) containing earnings dates, EPS estimates, and actual EPS
2. Download complete historical price data using yfinance
3. Calculate 2-day percentage changes for all historical dates: for each sequence of 3 consecutive trading days (Day 1, Day 2, Day 3), compute the return as Close_Day3 / Close_Day1 - 1. (Assume Day 2 may correspond to the earnings announcement.)
4. Identify positive earnings surprises (where "actual EPS > estimated EPS" OR "Surprise (%)>0")
5. Calculate 2-day percentage changes following positive earnings surprises
6. Compare the median 2-day percentage change for positive surprises vs. all historical dates

Context: Earnings announcements, especially when they exceed analyst expectations, can significantly impact stock prices in the short term.

Reference: Yahoo Finance earnings calendar - https://finance.yahoo.com/calendar/earnings?symbol=AMZN

*Additional*: Is there a correlation between the magnitude of the earnings surprise and the stock price reaction? Does the market react differently to earnings surprises during bull vs. bear markets?)

In [34]:
import pandas as pd
import yfinance as yf
import numpy as np

# Step 1: Load earnings data CSV
earnings_df = pd.read_csv("C:/Users/Sandra/Documents/GitHub/Stock-Markets-Analytics/HW1/ha1_Amazon.csv", delimiter=';')

# Make sure 'Date' is datetime type
earnings_df['Earnings Date'] = earnings_df['Earnings Date'].str.split(' at').str[0]
earnings_df['Earnings Date'] = pd.to_datetime(earnings_df['Earnings Date'], errors='coerce')

# Clean numeric columns: replace non-numeric placeholders with NaN, then convert
earnings_df['EPS Estimate'] = pd.to_numeric(earnings_df['EPS Estimate'].replace(['-', '???', '???'], np.nan), errors='coerce')
earnings_df['Reported EPS'] = pd.to_numeric(earnings_df['Reported EPS'].replace(['-', '???', '???'], np.nan), errors='coerce')

# Clean Surprise (%): remove '%' and '+' and convert to float
earnings_df['Surprise (%)'] = earnings_df['Surprise (%)'].str.replace('%', '', regex=False)
earnings_df['Surprise (%)'] = earnings_df['Surprise (%)'].str.replace('+', '', regex=False)
earnings_df['Surprise (%)'] = pd.to_numeric(earnings_df['Surprise (%)'], errors='coerce')

print(earnings_df.head())

# Step 2: Download full historical data for AMZN from earliest earnings date minus some buffer
start_date = earnings_df['Earnings Date'].min() - pd.Timedelta(days=30)
end_date = pd.Timestamp.today()

ticker = "AMZN"
price_data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

# Use Close prices for returns calculation
price_data = price_data[['Close']]

# Step 3: Calculate 2-day percentage changes for all historical dates
# We define 2-day return as (Close on Day 3 / Close on Day 1) - 1
# Flatten columns to avoid MultiIndex
price_data.columns = ['_'.join(filter(None, col)).strip() for col in price_data.columns.values]

# Now price_data.columns will look like ['Close_AMZN', '2d_return']
# Access the Close column with:
close_col = [col for col in price_data.columns if col.startswith('Close')][0]

price_data['2d_return'] = price_data[close_col].shift(-2) / price_data[close_col] - 1
price_data = price_data.dropna(subset=['2d_return'])

# Drop last two rows which will have NaN returns
price_data = price_data.dropna(subset=['2d_return'])

# Step 4: Identify positive earnings surprises
# Criteria: actual EPS > estimated EPS OR Surprise (%) > 0
positive_surprises = earnings_df[
    (earnings_df['Reported EPS'] > earnings_df['EPS Estimate']) | (earnings_df['Surprise (%)'] > 0)
]

# Step 5: For each positive earnings surprise date, find the corresponding 2-day return in price_data
# The earnings date might not be in price_data index if it's a non-trading day,
# so get the nearest next trading day after the earnings date to measure return starting that day
positive_returns = []

for edate in positive_surprises['Earnings Date']:
    # Find closest trading day on or after earnings date
    possible_dates = price_data.loc[edate:]
    if possible_dates.empty:
        continue
    nearest_date = possible_dates.index[0]
    ret = price_data.loc[nearest_date, '2d_return']
    positive_returns.append(ret)

positive_returns = pd.Series(positive_returns).dropna()

# Step 6: Compare median 2-day returns for positive surprises vs all dates
median_positive = positive_returns.median()
median_all = price_data['2d_return'].median()

print(f"Median 2-day return following positive earnings surprises: {median_positive:.4f} ({median_positive*100:.2f}%)")
print(f"Median 2-day return over all historical dates: {median_all:.4f} ({median_all*100:.2f}%)")

# Optional Additional: correlation between Surprise (%) and 2-day return
# Merge surprise data and 2-day returns by matching earnings date to nearest trading date
def get_2d_return_for_date(date):
    possible_dates = price_data.loc[date:]
    if possible_dates.empty:
        return np.nan
    return possible_dates['2d_return'].iloc[0]

positive_surprises['Nearest Return'] = positive_surprises['Earnings Date'].apply(get_2d_return_for_date)
positive_surprises.dropna(subset=['Nearest Return'], inplace=True)

correlation = positive_surprises['Surprise (%)'].corr(positive_surprises['Nearest Return'])
print(f"Correlation between Surprise (%) and 2-day return: {correlation:.4f}")


[*********************100%***********************]  1 of 1 completed

  Symbol           Company Earnings Date  EPS Estimate  Reported EPS  \
0   AMZN    Amazon.com Inc    2026-04-29           NaN           NaN   
1   AMZN    Amazon.com Inc    2026-02-04           NaN           NaN   
2   AMZN    Amazon.com Inc    2025-10-29           NaN           NaN   
3   AMZN    Amazon.com Inc    2025-07-30           NaN           NaN   
4   AMZN  Amazon.com, Inc.    2025-05-01           NaN           NaN   

   Surprise (%)  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4         16.74  
Median 2-day return following positive earnings surprises: 0.0027 (0.27%)
Median 2-day return over all historical dates: 0.0017 (0.17%)
Correlation between Surprise (%) and 2-day return: 0.1828



C:\Users\Sandra\AppData\Local\Temp\ipykernel_5280\656462027.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_surprises['Nearest Return'] = positive_surprises['Earnings Date'].apply(get_2d_return_for_date)
C:\Users\Sandra\AppData\Local\Temp\ipykernel_5280\656462027.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_surprises.dropna(subset=['Nearest Return'], inplace=True)


Median 2-day percentage change in AMZN stock price following positive surprise earnings days (4 points)
* 4.5
* 3.2
* 2.6
* 1.8

### Question 5.  [Exploratory, optional] Brainstorm potential idea for your capstone project

**Free text answer**

Describe the capstone project you would like to pursue, considering your aspirations, ML model predictions, and prior knowledge. Even if you are unsure at this stage, try to generate an idea you would like to explore-such as a specific asset class, country, industry vertical, or investment strategy. Be as specific as possible.

*Example: I want to build a short-term prediction model for the US/India/Brazil stock markets, focusing on the largest stocks over a 30-day investment horizon. I plan to use RSI and MACD technical indicators and news coverage data to generate predictions.*

Idea for your capstone project (1 point)

I plan to build a short-term predictive model for the US technology sector stocks, specifically targeting Amazon (AMZN), Apple (AAPL), and Microsoft (MSFT). The model will focus on forecasting 7-day forward returns immediately following earnings announcements. Inputs will include quantitative features such as historical price data, technical indicators (RSI, MACD, Bollinger Bands), and earnings surprise metrics (percentage surprise, EPS actual vs estimate). In addition, I will incorporate sentiment analysis derived from financial news articles and Twitter data within a 3-day window around earnings dates to capture market sentiment shifts.

The model architecture will explore gradient boosting methods (e.g., XGBoost) and transformer-based natural language processing for sentiment features. Furthermore, I intend to add a market regime detection component using volatility and macroeconomic indicators to adapt predictions during bull versus bear markets.

This project leverages my background in machine learning and finance, and aims to provide an actionable signal for swing trading strategies centered on earnings-driven volatility in large tech stocks.

### Question 6. [Exploratory, optional] Investigate new metrics

**Free text answer**

Using the data sources we have covered (or any others you find relevant), download and explore a few additional metrics or time series that could be valuable for your project. Briefly explain why you think each metric is useful. This does not need to be a comprehensive list-focus on demonstrating your ability to generate data requests based on your project description, identify and locate the necessary data, and explain how you would retrieve it using Python.

To enhance my capstone project on short-term earnings-driven stock return prediction for major US tech companies, I plan to explore the following additional metrics and time series:

1. Implied Volatility (IV) from Options Data:

IV reflects market expectations of future volatility around earnings announcements. Including IV could improve the model’s ability to anticipate price swings.

Data source & retrieval: Use the yfinance Python package or APIs like Polygon.io or IEX Cloud to download options chains data. For example, with yfinance:

ticker = yf.Ticker("AMZN")  
options_dates = ticker.options  
iv_data = ticker.option_chain(options_dates[0]).impliedVolatility  # Extract implied volatility for near-term expiry  

2. Trading Volume and Volume Spike Indicators:

Unusual volume spikes often accompany earnings surprises and can signal strong market reactions. Volume metrics will complement price and sentiment data.

Data source & retrieval: Volume data is part of historical price data available from yfinance:

price_data = yf.download("AMZN", start="2023-01-01", end="2024-01-01")[['Close', 'Volume']]  

3. Market Sentiment Scores from News and Social Media:

Extracting sentiment from financial news articles, analyst reports, or Twitter posts around earnings dates could capture investor mood and expectations not reflected in price alone.

Data source & retrieval: Use APIs such as NewsAPI, Twitter API (v2), or web scraping tools to collect text data, then apply NLP sentiment models like VADER or fine-tuned transformers (e.g., FinBERT).

Example with NewsAPI:

from newsapi import NewsApiClient  
newsapi = NewsApiClient(api_key='YOUR_API_KEY')  
articles = newsapi.get_everything(q='Amazon earnings', from_param='2025-04-28', to='2025-04-30')  

4. Macroeconomic Indicators (e.g., interest rates, inflation, consumer sentiment):

Broader economic conditions influence investor behavior and stock price reactions. Including these factors could help differentiate earnings reactions during different market regimes.

Data source & retrieval: Use fredapi for Federal Reserve Economic Data (FRED):

from fredapi import Fred  
fred = Fred(api_key='YOUR_FRED_API_KEY')  
cpi = fred.get_series('CPIAUCSL', start_date='2023-01-01')  

These metrics are valuable as they provide multidimensional signals—market expectations (IV), actual trading activity (volume), investor sentiment (news and social media), and macro environment context—which collectively could enhance prediction accuracy for earnings-related stock moves.

